### 2. Merge

#### take look at all the cleaned dataset in once 

In [65]:
import pandas as pd
import copy
collisions_cleaned = pd.read_csv('CleanedData/collisions_cleaned.csv')
intersections_cleaned  = pd.read_csv('CleanedData/intersections_cleaned.csv')
traffic_signals_cleaned  = pd.read_csv('CleanedData/traffic_signals_cleaned.csv')
poles_cleaned  = pd.read_csv('CleanedData/poles_cleaned.csv')
streets_cleaned  = pd.read_csv('CleanedData/streets_cleaned.csv')

In [25]:
collisions = copy.deepcopy(collisions_cleaned)
print("Rows:",len(collisions))
for column in collisions.columns:
    print(f"Column name: {column}, Type: {collisions[column].dtype}")
    #print(f"Unique values: {collisions[column].unique()}")

Rows: 79439
Column name: X, Type: float64
Column name: Y, Type: float64
Column name: ADDRTYPE, Type: object
Column name: SEVERITYCODE, Type: object
Column name: LIGHTCOND, Type: object
Column name: ROADCOND, Type: object
Column name: SPEEDING, Type: object
Column name: UNDERINFL, Type: object
Column name: INATTENTIONIND, Type: object


In [26]:
intersections = copy.deepcopy(intersections_cleaned)
print("Rows:",len(intersections))
for column in intersections.columns:
    print(f"Column name: {column}, Type: {intersections[column].dtype}")
    #print(f"Unique values: {intersections[column].unique()}")

#Arterial classification code:
#5 - Interstate Freeway
#4 - State Highway
#3 - Collector Arterial
#2 - Minor Arterial
#1 - Principal Arterial
#0 - Not Designated (not an arterial) 

Rows: 15469
Column name: X, Type: float64
Column name: Y, Type: float64
Column name: UNITDESC, Type: object
Column name: ARTERIALCLASSCD, Type: float64
Column name: SIGNAL_TYPE, Type: object


In [27]:
traffic_signals = copy.deepcopy(traffic_signals_cleaned)
print("Rows:",len(traffic_signals))
for column in traffic_signals.columns:
    print(f"Column name: {column}, Type: {traffic_signals[column].dtype}")
    #print(f"Unique values: {traffic_signals[column].unique()}")

Rows: 1134
Column name: UNITDESC, Type: object
Column name: SIGNAL_TYPE, Type: object
Column name: COMPTYPE, Type: int64
Column name: CONDITION, Type: object
Column name: ARTERIAL_CLASS, Type: float64
Column name: INT_SIGNAL_TYPE_CD, Type: object
Column name: PEDSIGNALYN, Type: object
Column name: PEDAUDIODEVICEYN, Type: object
Column name: PP_QUANTITY, Type: int64
Column name: PH_QUANTITY, Type: int64
Column name: BIKESIGNALHDYN, Type: object
Column name: LTTURNSIGNALYN, Type: object
Column name: RTTURNSIGNALYN, Type: object
Column name: DETDEVSTOPBARYN, Type: object
Column name: DETDEVADVANCEDYN, Type: object
Column name: DETDEVSYSTEMYN, Type: object
Column name: NUM_ATTACHMENTS, Type: int64


In [28]:
poles = copy.deepcopy(poles_cleaned)
print("Rows:",len(poles))
for column in poles.columns:
    print(f"Column name: {column}, Type: {poles[column].dtype}")
    #print(f"Unique values: {poles[column].unique()}")

Rows: 112427
Column name: Unnamed: 0, Type: int64
Column name: X, Type: float64
Column name: Y, Type: float64
Column name: HEIGHT, Type: int64
Column name: STREETLIGHT, Type: object


In [29]:
streets = copy.deepcopy(streets_cleaned)
print("Rows:",len(streets))
for column in streets.columns:
    print(f"Column name: {column}, Type: {streets[column].dtype}")
    #print(f"Unique values: {streets[column].unique()}")

Rows: 23721
Column name: Unnamed: 0, Type: int64
Column name: UNITDESC, Type: object
Column name: STNAME_ORD, Type: object
Column name: XSTRLO, Type: object
Column name: XSTRHI, Type: object
Column name: ARTCLASS, Type: float64
Column name: SPEEDLIMIT, Type: float64
Column name: SURFACETYPE_1, Type: object
Column name: PVMTCONDINDX1, Type: float64
Column name: PVMTCONDINDX2, Type: float64
Column name: TRANCLASS, Type: int64
Column name: SLOPE_PCT, Type: float64
Column name: SHAPE_Length, Type: float64


####  Some data visualization

In [ ]:
import folium
# plot all 15469 intersections
streetlights = poles[poles['STREETLIGHT'] == 'YES']

seattle_coordinates = (47.6062, -122.3321)
m = folium.Map(location=seattle_coordinates, zoom_start=12)


for index, row in intersections.iterrows():
    folium.Circle(
        location=[row['Y'], row['X']], 
        radius=15, 
        color='blue', 
        fill=0, 
        fill_color='blue', 
        fill_opacity=0.3, 
    ).add_to(m)

#plot all collision points
for index, row in collisions.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=2,  
        color='red',  
        fill=True,
        fill_color='red', 
        fill_opacity=1, 
    ).add_to(m)

#plot all poles points
for index, row in streetlights.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=2,  
        color='yellow',  
        fill=True,
        fill_color='yellow', 
        fill_opacity=1, 
    ).add_to(m)

m

In [ ]:
#Some formulas
import pandas as pd
from rtree import index
import math 
from IPython.display import display, clear_output

def haversine(lat1, lon1, lat2, lon2):
    # 将十进制度数转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # 地球半径 (单位: 米)
    earth_radius = 6371000
    distance = earth_radius * c
    return distance

def is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
    distance = haversine(latitude, longitude, center_latitude, center_longitude)
    return distance <= radius

def count_collisions(row, idx, collisions, radius):
    global progress_counter
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    center_latitude = row['Y']
    center_longitude = row['X']
    nearby_points = list(idx.intersection((center_longitude - radius, center_latitude - radius, center_longitude + radius, center_latitude + radius)))
    count = 0
    
    for point_idx in nearby_points:
        latitude = collisions.at[point_idx, 'Y']
        longitude = collisions.at[point_idx, 'X']
        if is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
            count += 1
    return count


### Merge Collision and Intersection

In [22]:
radius = 15
progress_counter = 0
idx = index.Index()
for i, row in collisions.iterrows():
    idx.insert(i, (row['X'], row['Y'], row['X'], row['Y']))

intersections_collision['collision_count'] = intersections.apply(lambda row: count_collisions(row, idx, collisions, radius), axis=1)

'Progress: 15469/15469'

In [23]:
intersections_collision.to_csv("MergedData/intersections_collisions.csv", index=False)
intersections_collision.head(10)

,X,Y,UNITDESC,ARTERIALCLASSCD,SIGNAL_TYPE,collision_count
0,-122.329732,47.523051,4TH AVE S AND S HENDERSON N ST,0.0,NONE,0
1,-122.371401,47.692058,12TH AVE NW AND NW 87TH ST,0.0,NONE,0
2,-122.289176,47.617639,34TH AVE AND E HOWELL ST,3.0,NONE,0
3,-122.320648,47.674000,NE 63RD ST AND NE RAVENNA EB BV,1.0,NONE,1
4,-122.263468,47.547165,WILSON AVE S AND S UPLAND RD,2.0,NONE,1
5,-122.360652,47.684740,3RD AVE NW AND NW 77TH ST,3.0,NONE,6
6,-122.305361,47.695749,RAVENNA AVE NE AND NE 92ND ST,1.0,NONE,14
7,-122.286434,47.599643,LAKESIDE UPPER AVE S AND S LESCHI PL,0.0,NONE,2
8,-122.280735,47.537049,42ND AVE S AND S OTHELLO ST,2.0,NONE,10
9,-122.296116,47.618899,M L KING JR WAY AND E DENNY WAY,3.0,NONE,19


### Merge with ploes

In [34]:
intersections_collision = pd.read_csv('MergedData/intersections_collisions.csv')
streetlights = poles[poles['STREETLIGHT'] == 'YES']

In [35]:
radius = 15
progress_counter = 0
streetlights_counts = []
streetlights_height_avgs = []

for index, row in intersections_collision.iterrows():
    center_latitude = row['Y']
    center_longitude = row['X']

    streetlights_within_radius = []
    
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    for _, streetlight in streetlights.iterrows():
        latitude = streetlight['Y']
        longitude = streetlight['X']

        if is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
            streetlights_within_radius.append(streetlight)

    streetlights_count = len(streetlights_within_radius)
    streetlights_counts.append(streetlights_count)

    streetlights_height_sum = sum([streetlight['HEIGHT'] for streetlight in streetlights_within_radius])
    streetlights_height_avg = streetlights_height_sum / streetlights_count if streetlights_count > 0 else 0
    streetlights_height_avgs.append(streetlights_height_avg)



'Progress: 15469/15469'

In [37]:
intersections_collision_streetlights = intersections_collision
intersections_collision_streetlights['streetlights'] = streetlights_counts
intersections_collision_streetlights['streetlights_height_avg'] = streetlights_height_avgs

intersections_collision_streetlights.to_csv("MergedData/intersections_collision_streetlights.csv", index=False)
intersections_collision_streetlights.head(10)

,X,Y,UNITDESC,ARTERIALCLASSCD,SIGNAL_TYPE,collision_count,streetlights,streetlights_height_avg
0,-122.329732,47.523051,4TH AVE S AND S HENDERSON N ST,0.0,NONE,0,0,0.0
1,-122.371401,47.692058,12TH AVE NW AND NW 87TH ST,0.0,NONE,0,1,50.0
2,-122.289176,47.617639,34TH AVE AND E HOWELL ST,3.0,NONE,0,2,50.0
3,-122.320648,47.674000,NE 63RD ST AND NE RAVENNA EB BV,1.0,NONE,1,1,0.0
4,-122.263468,47.547165,WILSON AVE S AND S UPLAND RD,2.0,NONE,1,1,50.0
5,-122.360652,47.684740,3RD AVE NW AND NW 77TH ST,3.0,NONE,6,1,50.0
6,-122.305361,47.695749,RAVENNA AVE NE AND NE 92ND ST,1.0,NONE,14,0,0.0
7,-122.286434,47.599643,LAKESIDE UPPER AVE S AND S LESCHI PL,0.0,NONE,2,1,55.0
8,-122.280735,47.537049,42ND AVE S AND S OTHELLO ST,2.0,NONE,10,2,33.0
9,-122.296116,47.618899,M L KING JR WAY AND E DENNY WAY,3.0,NONE,19,1,55.0


In [39]:
intersections_collision_streetlights = pd.read_csv('MergedData/intersections_collision_streetlights.csv')
#check how many different data values of poles_height_avg in intersections_collision_streetlights
unique_values = intersections_collision_streetlights['streetlights_height_avg'].nunique()
print(f'streetlights_height_avg has {unique_values} unique data value。')

streetlights_height_avg has 209 unique data value。


### Merge with streets

In [66]:
streets = copy.deepcopy(streets_cleaned)
print("Rows:",len(streets))
for column in streets.columns:
    print(f"Column name: {column}, Type: {streets[column].dtype}")
    #print(f"Unique values: {streets[column].unique()}")

Rows: 23721
Column name: Unnamed: 0, Type: int64
Column name: UNITDESC, Type: object
Column name: STNAME_ORD, Type: object
Column name: XSTRLO, Type: object
Column name: XSTRHI, Type: object
Column name: ARTCLASS, Type: float64
Column name: SPEEDLIMIT, Type: float64
Column name: SURFACETYPE_1, Type: object
Column name: PVMTCONDINDX1, Type: float64
Column name: PVMTCONDINDX2, Type: float64
Column name: TRANCLASS, Type: int64
Column name: SLOPE_PCT, Type: float64
Column name: SHAPE_Length, Type: float64


In [67]:
streets.head(10)

,Unnamed: 0,UNITDESC,STNAME_ORD,XSTRLO,XSTRHI,ARTCLASS,SPEEDLIMIT,SURFACETYPE_1,PVMTCONDINDX1,PVMTCONDINDX2,TRANCLASS,SLOPE_PCT,SHAPE_Length
0,0,1ST AVE BETWEEN SENECA ST AND UNIVERSITY ST,1ST AVE,SENECA ST,UNIVERSITY ST,2.0,25.0,PCC,87.0,62.0,1,4.0,305.966050
1,1,1ST AVE BETWEEN PIKE ST AND PINE ST,1ST AVE,PIKE ST,PINE ST,2.0,25.0,AC/PCC,57.0,58.0,1,5.0,426.031562
2,2,1ST AVE N BETWEEN VALLEY UPPER ST AND ALOHA ST,1ST AVE N,VALLEY UPPER ST,ALOHA ST,0.0,20.0,PCC,0.0,0.0,0,17.0,297.147592
3,3,1ST AVE N BETWEEN LYNN ST AND MCGRAW S ST,1ST AVE N,LYNN ST,MCGRAW S ST,0.0,20.0,AC,9.0,0.0,0,3.0,174.804983
4,4,1ST AVE N BETWEEN FULTON S ST AND FULTON N ST,1ST AVE N,FULTON S ST,FULTON N ST,0.0,20.0,PCC,0.0,0.0,0,5.0,73.110708
5,5,1ST AVE NE BETWEEN NE 60TH ST AND NE 61ST ST,1ST AVE NE,NE 60TH ST,NE 61ST ST,0.0,20.0,PCC,92.0,0.0,0,3.0,212.585996
6,6,1ST AVE NE BETWEEN N 122ND ST AND CORLISS S AVE N,1ST AVE NE,N 122ND ST,CORLISS S AVE N,3.0,25.0,ST,36.0,0.0,3,2.0,837.063532
7,7,1ST AVE NW BETWEEN N 42ND E ST AND NW 43RD ST,1ST AVE NW,N 42ND E ST,NW 43RD ST,0.0,20.0,PCC,65.0,0.0,0,5.0,677.122040
8,8,1ST AVE NW BETWEEN NW 53RD ST AND N 54TH ST,1ST AVE NW,NW 53RD ST,N 54TH ST,0.0,20.0,PCC,61.0,0.0,0,0.0,152.414888
9,9,1ST AVE NW BETWEEN NW 65TH ST AND NW 67TH ST,1ST AVE NW,NW 65TH ST,NW 67TH ST,0.0,20.0,PCC,48.0,0.0,0,2.0,649.833791


In [68]:
#check if the slope are all positive numbers
unique_values = streets['SLOPE_PCT'].unique()
print(unique_values)

[ 4.  5. 17.  3.  2.  0.  1.  6. 10.  7.  8.  9. 13. 11. 15. 14. 12. 16.
 24. 18. 21. 19. 20. 22. 25. 32. 23. 35. 33. 27. 38. 34. 47. 29.]


In [69]:
#check if the PVMTCONDINDX1 unique values
unique_values = streets['PVMTCONDINDX1'].unique()
print(unique_values)

[ 87.  57.   0.   9.  92.  36.  65.  61.  48.  71.  96.  91.  73.  56.
  70.  20.  37.  82.  68.  86.  81.  74.  19.  45.  55.  52.  69.  78.
  77.  38.  75.  62.  42.  60.  79.  88.  63.  58.  83.  47.  80.  66.
  33.  72. 100.  97.  41.  44.   5.  26.  21.  43.  12.  93.  90.  13.
  25.  46.  99.  84.   7.  24.  59.  76.  17.  11.  40.  27.  85.  16.
  89.  94.  31.  95.  54.  49.  29.  98.  51.   3.  22.  14.  67.  64.
  50.  15.  28.  34.  23.  39.  35.  30.  32.  10.   8.  53.   2.  18.
   6.   4.   1.]


In [70]:
#check How many 0s in PVMTCONDINDX1
value_counts = streets['PVMTCONDINDX1'].value_counts()
zero_count = value_counts.loc[0.0] if 0.0 in value_counts.index else 0
print("Number of 0.0 values:", zero_count)

Number of 0.0 values: 4253


In [128]:
#check How many 0s in PVMTCONDINDX2
value_counts = streets['PVMTCONDINDX2'].value_counts()
zero_count = value_counts.loc[0.0] if 0.0 in value_counts.index else 0
print("Number of 0.0 values:", zero_count)

Number of 0.0 values: 23039


In [71]:
intersections_collision_streetlights = pd.read_csv('MergedData/intersections_collision_streetlights.csv')

print("Rows:",len(intersections_collision_streetlights))
for column in intersections_collision_streetlights.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights[column].dtype}")
    
intersections_collision_streetlights.head(10)

Rows: 15469
Column name: X, Type: float64
Column name: Y, Type: float64
Column name: UNITDESC, Type: object
Column name: ARTERIALCLASSCD, Type: float64
Column name: SIGNAL_TYPE, Type: object
Column name: collision_count, Type: int64
Column name: streetlights, Type: int64
Column name: streetlights_height_avg, Type: float64


,X,Y,UNITDESC,ARTERIALCLASSCD,SIGNAL_TYPE,collision_count,streetlights,streetlights_height_avg
0,-122.329732,47.523051,4TH AVE S AND S HENDERSON N ST,0.0,NONE,0,0,0.0
1,-122.371401,47.692058,12TH AVE NW AND NW 87TH ST,0.0,NONE,0,1,50.0
2,-122.289176,47.617639,34TH AVE AND E HOWELL ST,3.0,NONE,0,2,50.0
3,-122.320648,47.674000,NE 63RD ST AND NE RAVENNA EB BV,1.0,NONE,1,1,0.0
4,-122.263468,47.547165,WILSON AVE S AND S UPLAND RD,2.0,NONE,1,1,50.0
5,-122.360652,47.684740,3RD AVE NW AND NW 77TH ST,3.0,NONE,6,1,50.0
6,-122.305361,47.695749,RAVENNA AVE NE AND NE 92ND ST,1.0,NONE,14,0,0.0
7,-122.286434,47.599643,LAKESIDE UPPER AVE S AND S LESCHI PL,0.0,NONE,2,1,55.0
8,-122.280735,47.537049,42ND AVE S AND S OTHELLO ST,2.0,NONE,10,2,33.0
9,-122.296116,47.618899,M L KING JR WAY AND E DENNY WAY,3.0,NONE,19,1,55.0


In [72]:
unique_values = intersections_collision_streetlights['UNITDESC'].unique()
print(len(unique_values))

15468


In [73]:
def split_unitdesc(unitdesc):
    streets = unitdesc.split(' AND ')
    return pd.Series([streets[0], streets[1]])

if 'UNITDESC' in intersections_collision_streetlights.columns:
    intersections_collision_streetlights[['Street1', 'Street2']] = intersections_collision_streetlights['UNITDESC'].apply(split_unitdesc)
else:
    print("Error: 'UNITDESC' column not found in the DataFrame")

In [74]:
intersections_collision_streetlights.head(10)

,X,Y,UNITDESC,ARTERIALCLASSCD,SIGNAL_TYPE,collision_count,streetlights,streetlights_height_avg,Street1,Street2
0,-122.329732,47.523051,4TH AVE S AND S HENDERSON N ST,0.0,NONE,0,0,0.0,4TH AVE S,S HENDERSON N ST
1,-122.371401,47.692058,12TH AVE NW AND NW 87TH ST,0.0,NONE,0,1,50.0,12TH AVE NW,NW 87TH ST
2,-122.289176,47.617639,34TH AVE AND E HOWELL ST,3.0,NONE,0,2,50.0,34TH AVE,E HOWELL ST
3,-122.320648,47.674000,NE 63RD ST AND NE RAVENNA EB BV,1.0,NONE,1,1,0.0,NE 63RD ST,NE RAVENNA EB BV
4,-122.263468,47.547165,WILSON AVE S AND S UPLAND RD,2.0,NONE,1,1,50.0,WILSON AVE S,S UPLAND RD
5,-122.360652,47.684740,3RD AVE NW AND NW 77TH ST,3.0,NONE,6,1,50.0,3RD AVE NW,NW 77TH ST
6,-122.305361,47.695749,RAVENNA AVE NE AND NE 92ND ST,1.0,NONE,14,0,0.0,RAVENNA AVE NE,NE 92ND ST
7,-122.286434,47.599643,LAKESIDE UPPER AVE S AND S LESCHI PL,0.0,NONE,2,1,55.0,LAKESIDE UPPER AVE S,S LESCHI PL
8,-122.280735,47.537049,42ND AVE S AND S OTHELLO ST,2.0,NONE,10,2,33.0,42ND AVE S,S OTHELLO ST
9,-122.296116,47.618899,M L KING JR WAY AND E DENNY WAY,3.0,NONE,19,1,55.0,M L KING JR WAY,E DENNY WAY


In [131]:
import numpy as np

def find_related_streets(row):

    match_count = ((streets['STNAME_ORD'] == row['Street1']) | (streets['STNAME_ORD'] == row['Street2'])).astype(int) + \
              ((streets['XSTRLO'] == row['Street1']) | (streets['XSTRLO'] == row['Street2'])).astype(int) + \
              ((streets['XSTRHI'] == row['Street1']) | (streets['XSTRHI'] == row['Street2'])).astype(int)

    ge_2_indices = match_count[match_count >= 2].index
    ge_2_indices_list = ge_2_indices.tolist()
    related_streets = streets.loc[ge_2_indices_list]

    return related_streets

def calculate_average_features(row):
    global progress_counter
    
    related_streets = find_related_streets(row)
    
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    if not related_streets.empty:
        return pd.Series([related_streets.shape[0],
                          related_streets['PVMTCONDINDX1'].mean(),
                          related_streets['PVMTCONDINDX2'].mean(),
                          related_streets['SLOPE_PCT'].mean(),
                          related_streets['TRANCLASS'].mean(),
                          related_streets['SPEEDLIMIT'].mean()])
    else:
        return pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan])

In [132]:
progress_counter = 0
intersections_collision_streetlights_streets = copy.deepcopy(intersections_collision_streetlights)
intersections_collision_streetlights_streets[['related_street_num','PVMTCONDINDX1_mean', 'PVMTCONDINDX2_mean', 'SLOPE_PCT_mean', 'TRANCLASS_mean', 'SPEEDLIMIT_mean']] = intersections_collision_streetlights_streets.apply(calculate_average_features, axis=1)

'Progress: 15469/15469'

In [127]:
intersections_collision_streetlights_streets.head(10)

,X,Y,UNITDESC,ARTERIALCLASSCD,SIGNAL_TYPE,collision_count,streetlights,streetlights_height_avg,Street1,Street2,related_street_num,PVMTCONDINDX1_mean,PVMTCONDINDX2_mean,SLOPE_PCT_mean,TRANCLASS_mean,SPEEDLIMIT_mean
0,-122.329732,47.523051,4TH AVE S AND S HENDERSON N ST,0.0,NONE,0,0,0.0,4TH AVE S,S HENDERSON N ST,3.0,56.333333,0.0,1.333333,0.666667,23.333333
1,-122.371401,47.692058,12TH AVE NW AND NW 87TH ST,0.0,NONE,0,1,50.0,12TH AVE NW,NW 87TH ST,3.0,95.333333,0.0,1.666667,0.000000,20.000000
2,-122.289176,47.617639,34TH AVE AND E HOWELL ST,3.0,NONE,0,2,50.0,34TH AVE,E HOWELL ST,4.0,43.000000,0.0,7.250000,1.500000,22.500000
3,-122.320648,47.674000,NE 63RD ST AND NE RAVENNA EB BV,1.0,NONE,1,1,0.0,NE 63RD ST,NE RAVENNA EB BV,1.0,78.000000,0.0,0.000000,0.000000,20.000000
4,-122.263468,47.547165,WILSON AVE S AND S UPLAND RD,2.0,NONE,1,1,50.0,WILSON AVE S,S UPLAND RD,3.0,43.333333,0.0,4.000000,0.000000,23.333333
5,-122.360652,47.684740,3RD AVE NW AND NW 77TH ST,3.0,NONE,6,1,50.0,3RD AVE NW,NW 77TH ST,4.0,77.750000,0.0,2.000000,0.000000,22.500000
6,-122.305361,47.695749,RAVENNA AVE NE AND NE 92ND ST,1.0,NONE,14,0,0.0,RAVENNA AVE NE,NE 92ND ST,5.0,59.600000,0.0,3.200000,1.800000,23.000000
7,-122.286434,47.599643,LAKESIDE UPPER AVE S AND S LESCHI PL,0.0,NONE,2,1,55.0,LAKESIDE UPPER AVE S,S LESCHI PL,2.0,95.000000,0.0,6.000000,0.000000,20.000000
8,-122.280735,47.537049,42ND AVE S AND S OTHELLO ST,2.0,NONE,10,2,33.0,42ND AVE S,S OTHELLO ST,2.0,81.000000,0.0,1.500000,3.000000,25.000000
9,-122.296116,47.618899,M L KING JR WAY AND E DENNY WAY,3.0,NONE,19,1,55.0,M L KING JR WAY,E DENNY WAY,3.0,39.000000,0.0,6.333333,1.000000,21.666667


In [136]:
print("Rows:",len(intersections_collision_streetlights_streets))
for column in intersections_collision_streetlights_streets.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets[column].dtype},NAs,{len(intersections_collision_streetlights_streets[column])-len(intersections_collision_streetlights_streets[column].dropna())}")
    

Rows: 15469
Column name: X, Type: float64,NAs,0
Column name: Y, Type: float64,NAs,0
Column name: UNITDESC, Type: object,NAs,0
Column name: ARTERIALCLASSCD, Type: float64,NAs,0
Column name: SIGNAL_TYPE, Type: object,NAs,0
Column name: collision_count, Type: int64,NAs,0
Column name: streetlights, Type: int64,NAs,0
Column name: streetlights_height_avg, Type: float64,NAs,0
Column name: Street1, Type: object,NAs,0
Column name: Street2, Type: object,NAs,0
Column name: related_street_num, Type: float64,NAs,0
Column name: PVMTCONDINDX1_mean, Type: float64,NAs,499
Column name: PVMTCONDINDX2_mean, Type: float64,NAs,499
Column name: SLOPE_PCT_mean, Type: float64,NAs,499
Column name: TRANCLASS_mean, Type: float64,NAs,499
Column name: SPEEDLIMIT_mean, Type: float64,NAs,499


In [137]:
#Drop NAs
intersections_collision_streetlights_streets.dropna(inplace=True)

print("Rows:",len(intersections_collision_streetlights_streets))
for column in intersections_collision_streetlights_streets.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets[column].dtype},NAs,{len(intersections_collision_streetlights_streets[column])-len(intersections_collision_streetlights_streets[column].dropna())}")
    

Rows: 14970
Column name: X, Type: float64,NAs,0
Column name: Y, Type: float64,NAs,0
Column name: UNITDESC, Type: object,NAs,0
Column name: ARTERIALCLASSCD, Type: float64,NAs,0
Column name: SIGNAL_TYPE, Type: object,NAs,0
Column name: collision_count, Type: int64,NAs,0
Column name: streetlights, Type: int64,NAs,0
Column name: streetlights_height_avg, Type: float64,NAs,0
Column name: Street1, Type: object,NAs,0
Column name: Street2, Type: object,NAs,0
Column name: related_street_num, Type: float64,NAs,0
Column name: PVMTCONDINDX1_mean, Type: float64,NAs,0
Column name: PVMTCONDINDX2_mean, Type: float64,NAs,0
Column name: SLOPE_PCT_mean, Type: float64,NAs,0
Column name: TRANCLASS_mean, Type: float64,NAs,0
Column name: SPEEDLIMIT_mean, Type: float64,NAs,0


In [ ]:
columns_to_drop = ['Street1', 'Street2', 'column3']
intersections_collision_streetlights_streets.drop(columns=columns_to_drop, inplace=True)

In [138]:
intersections_collision_streetlights_streets.to_csv("MergedData/intersections_collision_streetlights_streets.csv", index=False)